In [54]:
from tqdm import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from helper_functions import accuracy_fn
from safetensors.torch import save_model

In [45]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

### Data Processing

In [46]:
data_transforms = transforms.Compose([
    transforms.ToTensor()
])

In [48]:
train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=data_transforms
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=data_transforms
)

In [ ]:
batch_size = 16

# put custom dataset to dataloader
train_dl = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [ ]:
# get how many classes are there
train_data.classes, len(train_data.classes)

(['0 - zero',
  '1 - one',
  '2 - two',
  '3 - three',
  '4 - four',
  '5 - five',
  '6 - six',
  '7 - seven',
  '8 - eight',
  '9 - nine'],
 10)

In [ ]:
# check shapes of dataloader
feature, label = next(iter(train_dl))
feature.shape, label.shape

(torch.Size([16, 1, 28, 28]), torch.Size([16]))

### Create Model

In [ ]:
class TinyVGG(nn.Module):

    def __init__(self):
        super().__init__()

        self.block_1 = nn.Sequential(
            nn.Conv2d(1, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.block_2 = nn.Sequential(
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(10, 10, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(10*7*7, len(train_data.classes)+1) # 7 * 7 because maxpool has been done twice which divides the shape of image by 2 twice
        )

    def forward(self, x):
        x = self.block_1(x)
        x = self.block_2(x)
        x = self.classifier(x)
        return x

model = TinyVGG().to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

### Train Model

In [ ]:
torch.manual_seed(20)

epochs = 5

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}\n------")
    
    # TRAINING
    train_loss, train_acc = 0, 0
    model.train()
    for batch, (X, y) in enumerate(train_dl):

        X, y = X.to(device), y.to(device)

        # forward pass
        train_pred = model(X)

        # metrics
        loss = loss_fn(train_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y_true=y, y_pred=train_pred.argmax(dim=1))

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # print metrics
    train_loss /= len(train_dl)
    train_acc /= len(train_dl)
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.2f}%")

    
    # TESTING
    test_loss, test_acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in test_dl:

            X, y = X.to(device), y.to(device)


            # forward pass
            test_pred = model(X)

            # metrics
            test_loss += loss_fn(test_pred, y)
            test_acc += accuracy_fn(y_true=y, y_pred=test_pred.argmax(dim=1))
        
        # print metrics
        test_loss /= len(test_dl)
        test_acc /= len(test_dl)
        print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.2f}%")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0
------
Train Loss: 0.7405 | Train Accuracy: 73.50%


 20%|██        | 1/5 [00:24<01:38, 24.68s/it]

Test Loss: 0.1074 | Test Accuracy: 96.68%
Epoch: 1
------
Train Loss: 0.1069 | Train Accuracy: 96.67%


 40%|████      | 2/5 [00:49<01:14, 24.86s/it]

...

Test Loss: 0.0618 | Test Accuracy: 98.08%
Epoch: 3
------
Train Loss: 0.0682 | Train Accuracy: 97.83%


 80%|████████  | 4/5 [01:37<00:24, 24.33s/it]

Test Loss: 0.0563 | Test Accuracy: 98.16%
Epoch: 4
------
Train Loss: 0.0595 | Train Accuracy: 98.15%


100%|██████████| 5/5 [02:01<00:00, 24.38s/it]

Test Loss: 0.0555 | Test Accuracy: 98.18%


### Save Model

In [52]:
model.state_dict()

OrderedDict([('block_1.0.weight',
              tensor([[[[ 0.2219, -0.2946,  0.1779],
                        [-0.1194, -0.2173, -0.3280],
                        [-0.3215, -0.0025,  0.0225]]],
              
              
                      [[[ 0.3302,  0.6183,  0.4863],
                        [ 0.0015,  0.8348,  0.4380],
                        [ 0.3063,  0.6007,  0.2562]]],
              
              
                      [[[ 0.1354,  0.3671,  0.3166],
                        [ 0.2871, -0.0690,  0.2459],
                        [-0.0434, -0.1520, -0.2511]]],
              
              
                      [[[ 0.3393, -0.1139,  0.2549],
                        [ 0.1675, -0.2209,  0.0379],
                        [-0.3073,  0.0071,  0.2797]]],
              
              
                      [[[-0.0090, -0.3136,  0.1748],
                        [ 0.0380,  0.0413,  0.0544],
                        [-0.2809, -0.2843, -0.3059]]],
              
              
           

In [55]:
save_model(model, "tinyvgg.safetensors")